# 🏗️ Arquitectura Medallón – Proyecto de Detección de Fraude

Este proyecto implementa la **Arquitectura Medallón** usando **Databricks Free Edition** y **Delta Lake**, organizando los datos en tres capas.

---

## 🥉 Bronze – Datos Crudos  
📥 Ingesta directa de archivos **CSV** sin transformaciones.  
Los datos se cargan tal como llegan mediante **Data Ingestion**.

---

## 🥈 Silver – Datos Procesados  
🧹 Limpieza, transformación e integración de las tablas  
(tarjetas, clientes, comercios, dispositivos y transacciones).

---

## 🥇 Gold – Datos Analíticos  
📊 Datos listos para análisis y visualización.  
Se generan métricas e indicadores para la **detección de fraude**.



---

## 🥉 Bronze – Datos Crudos

In [0]:
cards_bronze = spark.table("workspace.default.cards")
customers_bronze = spark.table("workspace.default.customers")
merchants_bronze = spark.table("workspace.default.merchants")
devices_bronze = spark.table("workspace.default.devices")
transactions_bronze = spark.table("workspace.default.transactions")

## 🥈 Silver – Datos Procesados  

In [0]:
from pyspark.sql.functions import col, to_timestamp

In [0]:
# TABLA CARDS
cards_silver = (
    cards_bronze
    .filter(col("card_id").isNotNull())
    .withColumn("limite_credito", col("limite_credito").cast("double"))
    .withColumn("fecha_emision", to_timestamp("fecha_emision"))
    .withColumn("fecha_vencimiento", to_timestamp("fecha_vencimiento"))
)

In [0]:
# TABLA CUSTOMERS
customers_silver = (
    customers_bronze
    .filter(col("customer_id").isNotNull())
)

In [0]:
# TABLA MERCHANTS
merchants_silver = (
    merchants_bronze
    .filter(col("merchant_id").isNotNull())
)

In [0]:
# TABLA DEVICES
devices_silver = (
    devices_bronze
    .filter(col("device_id").isNotNull())
)

In [0]:
# TABLA TRANSACTIONS
transactions_silver = (
    transactions_bronze
    .filter(col("transaction_id").isNotNull())
    .withColumn("monto", col("monto").cast("double"))
    .withColumn("fecha", to_timestamp("fecha"))
)

In [0]:
cards_silver.write.mode("overwrite").saveAsTable("workspace.default.cards_silver")
customers_silver.write.mode("overwrite").saveAsTable("workspace.default.customers_silver")
merchants_silver.write.mode("overwrite").saveAsTable("workspace.default.merchants_silver")
devices_silver.write.mode("overwrite").saveAsTable("workspace.default.devices_silver")
transactions_silver.write.mode("overwrite").saveAsTable("workspace.default.transactions_silver")

## 🥇 Gold – Datos Analíticos  

In [0]:
# CARGA DE TABLAS DE LA CAPA SILVER
cards_silver = spark.table("workspace.default.cards_silver")
customers_silver = spark.table("workspace.default.customers_silver")
merchants_silver = spark.table("workspace.default.merchants_silver")
devices_silver = spark.table("workspace.default.devices_silver")
transactions_silver = spark.table("workspace.default.transactions_silver")

In [0]:
# DIMENSIONES LIMPIAS (sin IDs cruzados)
cards_dim = cards_silver.drop("customer_id")
customers_dim = customers_silver
merchants_dim = merchants_silver
devices_dim = devices_silver.drop("customer_id")

In [0]:
# CONSTRUCCIÓN DE LA TABLA GOLD
fraude_gold = (
    transactions_silver
    .join(cards_dim, "card_id", "left")
    .join(customers_dim, "customer_id", "left")
    .join(merchants_dim, "merchant_id", "left")
    .join(devices_dim, "device_id", "left")
)

In [0]:
# COLUMNA ANALÍTICA
from pyspark.sql.functions import when, col

fraude_gold = fraude_gold.withColumn(
    "es_fraude",
    when(col("monto") > 5000, 1).otherwise(0)
)

In [0]:
fraude_gold.printSchema()

root
 |-- device_id: long (nullable = true)
 |-- merchant_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- card_id: long (nullable = true)
 |-- transaction_id: long (nullable = true)
 |-- monto: double (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- ciudad_transaccion: string (nullable = true)
 |-- es_fraude: long (nullable = true)
 |-- motivo_fraude: string (nullable = true)
 |-- tipo_tarjeta: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- limite_credito: double (nullable = true)
 |-- fecha_emision: timestamp (nullable = true)
 |-- fecha_vencimiento: timestamp (nullable = true)
 |-- dni: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- edad: long (nullable = true)
 |-- ciudad_residencia: string (nullable = true)
 |-- fecha_registro: timestamp (nullable = true)
 |-- nivel_riesgo: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- rubro: string (nullable = true)
 |-- riesgo_comercio: string (nu

In [0]:
fraude_gold.write.mode("overwrite").saveAsTable(
    "workspace.default.fraude_gold"
)

In [0]:
spark.table("workspace.default.fraude_gold").count()

200000